In [8]:
import apogee.tools.read as apread
from apogee.tools import bitmask
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import window as wn
import os
import access_spectrum as acs
import reduce_dataset as rd
import polyfit as pf
import matplotlib
import apogee.spec.plot as splot
from read_clusterdata import read_caldata
# Show plots in notebook
%pylab inline
# Read in red clump sample
data = read_caldata()
data['APOGEE_ID'] = data['ID']
# Set random seed value
seedval = 1
# Start random seed
np.random.seed(seedval) #original was 17
# Set global plot font preferences
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

Populating the interactive namespace from numpy and matplotlib


In [9]:
direc = './open_clusters/pickles/'
pdirec = './open_clusters/fitplots/'
rdirec = './open_clusters/residuals_plots/'
specsfile = direc+'specs_cdata.pkl'
errsfile = direc+'errs_cdata.pkl'
ngen = False
if os.path.isfile(specsfile) and not ngen:
    specs,ind = acs.pklread(specsfile)
    errs,ind = acs.pklread(errsfile)
else:
    specdata = acs.get_spectra_asp(data,header = False)
    errdata= acs.get_spectra_asp(data,ext = 2, header = False)
    acs.pklwrite(specsfile,specdata)
    acs.pklwrite(errsfile,errdata)
    specs,ind = specdata
    errs,ind = errdata  
    print 'Missing {0} stars of {1}'.format(len(ind),len(specs))

In [4]:
len(specs)

484

In [10]:
elems = ['Al','Ca','C','Fe','K','Mg','Mn','Na','Ni','N','O','Si','S','Ti','V']
wins = {}
ths = {}
elemwindow = {}
totalw = np.zeros(len(specs[0]))
for elem in elems:
    w = wn.read(elem,dr = 12,apStarWavegrid=False)
    th = wn.tophat(elem,dr=12,apStarWavegrid=False)
    wins[elem] = w
    ths[elem] = w
    # Record which pixels are relevant for this element
    elemwindow[elem] = np.where(th==True)[0]
    totalw += w

In [11]:
order = 2
data = data[ind]
Tkeep = np.where(data['TEFF']!=0)
data = data[Tkeep]
specs = specs[Tkeep]
errs = errs[Tkeep]
clusters = np.unique(data['CLUSTER'])
maskdata = acs.get_spectra_ap(data,ext = 3, header = False,indx = 1)

In [18]:
doplot = True
badcombpixmask= bitmask.badpixmask()+2**bitmask.apogee_pixmask_int("SIG_SKYLINE")
def outplotname(cluster,pix):
    return '{0}_pix{1}fit_order{2}.png'.format(cluster,pix,order)

def cresid_elemname(cluster,elem):
    return '{0}_{1}_residuals_order{2}.pkl'.format(elem,cluster,order)

def cgerr_elemname(cluster,elem):
    return '{0}_{1}_cgerr_order{2}_seed{3}.pkl'.format(elem,cluster,order,seedval)

# Name the plot for each element
def coutplotname(clusterelem):
    return '{0}_{1}_residuals_order{2}_seed{3}.png'.format(elem,cluster,order,seedval)    

for cluster in clusters:
    ressname = '{0}_cresiduals_order{1}.pkl'.format(cluster,order)
    gname = '{0}_cgerr_order{1}_seed{2}.pkl'.format(cluster,order,seedval)
    disname = '{0}_discard_cresiduals_order{1}.pkl'.format(cluster,order)
    failname = '{0}_fail_cresidual_order{1}.pkl'.format(cluster,order)
    paramnane = '{0}_fitparam_order{1}.pkl'.format(cluster,order)
    Ckeep = np.where(data['CLUSTER']==cluster)
    cdata = data[Ckeep]
    cspecs = specs[Ckeep]
    cerrs = errs[Ckeep]
    ngen = False
    if os.path.isfile(direc+ressname) and os.path.isfile(direc+gname) and ngen:
        ress = acs.pklread(direc+ressname)
        gerr = acs.pklread(direc+gname)
    elif not os.path.isfile(direc+ressname) or os.path.isfile(direc+gname) or not ngen:
        ps = []
        ress = []
        discard = []
        gerr = []
        fails = []
        for pix in range(len(specs[0])):
            mask,badspec = rd.pixmask_find(maskdata[Ckeep],pix,bitval = badcombpixmask)
            mdata = cdata[mask]
            mspecs = cspecs[mask]
            merrs = cerrs[mask]
            Teff = np.arange(min(mdata['TEFF']),max(mdata['TEFF']))
            discard.append('{0} of {1}'.format(len(cdata)-len(mask[0]),len(cdata)))
            res = np.zeros(len(specs))
            erarr = np.zeros(len(specs))
            try:
                p = pf.regfit(mdata['TEFF'],mspecs[:,pix],err = merrs[:,pix],order = order)
                ps.append(p)
                res = mspecs[:,pix] - pf.poly(p,mdata['TEFF'],order = order)
                ress.append(res)
                if totalw[pix] != 0 and doplot:
                    pl = np.where((merrs[:,pix] < 0.1) & (mspecs[:,pix] != 0))
                    rpl = np.where((merrs[:,pix] > 0.1) & (mspecs[:,pix] != 0))
                    plt.figure(figsize = (12,10))
                    plt.subplot2grid((3,1),(0,0))
                    plt.errorbar(mdata['TEFF'][pl],mspecs[:,pix][pl],yerr = merrs[:,pix][pl],fmt='.')
                    plt.plot(Teff,pf.poly(p,Teff,order = order),linewidth = 3)
                    plt.plot(mdata['TEFF'][rpl],mspecs[:,pix][rpl],'.')
                    plt.xlabel('T_EFF')
                    plt.ylabel('Pixel {0}'.format(pix))
                    plt.subplot2grid((3,3),(1,0))
                    plt.plot(mdata['TEFF'][pl],res[pl],'.')
                    plt.xlabel('T_EFF')
                    plt.ylabel('Fit residuals')
                    plt.subplot2grid((3,3),(2,0))
                    plt.semilogx(errs[:,pix][pl],res[pl],'.')
                    plt.xlabel('Uncertainty in Pixel {0}'.format(pix))
                    plt.ylabel('Fit residuals')
                    print pdirec+outplotname(cluster,pix)
                    plt.savefig(pdirec+outplotname(cluster,pix))
                    plt.close()
            except LinAlgError:
                fails.append('Failure at pixel {0}'.format(pix))
                continue
            for mval in mask[0]:
                erarr[mask[0][mval]] = np.random.normal(0,errs[mval][pix])
            gerr.append(erarr)
        ress = np.array(ress)
        gerr = np.array(gerr)
        dfile = open(direc+'dis_'+ressname.split('.pkl')[0]+'.txt','wb')
        dfile.write("\n".join(discard))
        acs.pklwrite(direc+ressname,ress) 
        acs.pklwrite(direc+gname,gerr)
        acs.pklwrite(direc+paramname,ps)
        acs.pklwrite(direc+failname,fails)

    # Cycle through all elements, checking for weighted residual files, creating them if necessary
    for elem in elems:
        if os.path.isfile(direc+cresid_elemname(cluster,elem)) and os.path.isfile(direc+cgerr_elemname(cluster,elem)) and ngen:
            continue
        elif not os.path.isfile(direc+cresid_elemname(cluster,elem)) or not os.path.isfile(direc+cgerr_elemname(cluster,elem)) or not ngen:
            # Read in window associated with the element
            w = wins[elem]
            th = ths[elem]
            # Normalized weights
            nw = pf.normweights(w)[th]
            # Prepare lists to hold the weighted residual for each star for this element
            resids = []
            gresid = []
            for star in range(len(ress[0])):
                # Create weighted residual sum for each star
                resids.append(pf.genresidual(nw,ress[:,star][th]))
                gresid.append(pf.genresidual(nw,gerr[:,star][th]))
            resids = np.array(resids)
            gresid = np.array(gresid)
            # Save weighted residuals for all stars for this element
            acs.pklwrite(direc+cresid_elemname(cluster,elem),resids)
            acs.pklwrite(direc+cgerr_elemname(cluster,elem),gresid)

    # Plot residuals for each element

    # Make residual plots
    if doplot:
        for elem in elems:
            resids = acs.pklread(direc+cresid_elemname(cluster,elem))
            gresid = acs.pklread(direc+cgerr_elemname(cluster,elem))
            plt.figure(figsize = (12,10))
            allow = np.where(np.isnan(gresid) == False)

            # Plot residual histogram
            plt.subplot(121)
            plt.hist(resids[allow],bins = 20,range = (-0.1,0.1),stacked=True)
            plt.xlim(-0.1,0.1)
            plt.xlabel('Fit residual')
            plt.ylabel('Star Count')
            plt.suptitle('Uncertainty from $T_{eff}$, $\log(g)$, $Z$ fit for '+elem)

            # Plot flux uncertainty historgram
            plt.subplot(122)
            plt.hist(gresid[allow],bins = 40,range = (-0.1,0.1),stacked=True)
            plt.xlabel('Flux uncertainty'.format(elem))
            plt.ylabel('Star Count')
            plt.xlim(-0.1,0.1)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.savefig(rdirec+coutplotname(elem))

./open_clusters/fitplots/M107_pix0fit_order2.png
./open_clusters/fitplots/M107_pix1fit_order2.png
./open_clusters/fitplots/M107_pix2fit_order2.png
./open_clusters/fitplots/M107_pix5fit_order2.png
./open_clusters/fitplots/M107_pix13fit_order2.png
./open_clusters/fitplots/M107_pix15fit_order2.png
./open_clusters/fitplots/M107_pix16fit_order2.png
./open_clusters/fitplots/M107_pix17fit_order2.png
./open_clusters/fitplots/M107_pix18fit_order2.png
./open_clusters/fitplots/M107_pix19fit_order2.png
./open_clusters/fitplots/M107_pix20fit_order2.png
./open_clusters/fitplots/M107_pix21fit_order2.png
./open_clusters/fitplots/M107_pix22fit_order2.png
./open_clusters/fitplots/M107_pix23fit_order2.png
./open_clusters/fitplots/M107_pix24fit_order2.png
./open_clusters/fitplots/M107_pix25fit_order2.png
./open_clusters/fitplots/M107_pix26fit_order2.png
./open_clusters/fitplots/M107_pix29fit_order2.png
./open_clusters/fitplots/M107_pix32fit_order2.png
./open_clusters/fitplots/M107_pix192fit_order2.png
./o

NameError: name 'paramname' is not defined